<a href="https://colab.research.google.com/github/sipocz/Blood/blob/main/Blood_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [2]:
fname_url="https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip"
fname=fname_url.split("/")[-1]

In [3]:
!wget $fname_url


--2021-11-26 21:11:07--  https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip [following]
--2021-11-26 21:11:07--  https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16979362 (16M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  16.19M  --.-KB/s    in 0.1s    

2021-11-26 21:11:08 (114 MB/s) - ‘train.zi

In [4]:
!mkdir "train"
!rm ./train/Train.csv

rm: cannot remove './train/Train.csv': No such file or directory


In [5]:
!unzip $fname -d "./train"
!rm train.zip

Archive:  train.zip
  inflating: ./train/Train.csv       


In [6]:
fname_train="./train/Train.csv"
df=pd.read_csv(fname_train)

In [7]:
df.tail(25)

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
13115,ID_BQX9N2R9,0.527403,0.533425,0.535697,0.538119,0.548961,0.552276,0.557599,0.562819,0.570729,0.586204,0.598432,0.607677,0.612164,0.616941,0.618651,0.618925,0.619145,0.616675,0.614034,0.607355,0.603343,0.599383,0.594927,0.592300,0.588607,0.585589,0.583024,0.580153,0.579698,0.578190,0.577065,0.577985,0.578447,0.579168,0.581671,0.584603,0.587384,0.591071,0.595397,...,1.556209,1.549685,1.537662,1.535926,1.511143,1.503861,1.493837,1.478955,1.464024,1.471290,1.448233,1.437791,1.433865,1.428090,1.416801,1.406560,1.396301,1.402107,1.386420,1.376283,1.379693,1.363067,1.368216,1.360280,1.365332,1.365070,1.371049,1.365990,1.372199,1.376005,1.359063,1.371432,1.364208,1.409573,1.347664,42.57,19.51,ok,ok,ok
13116,ID_AERQBMHL,0.468074,0.471415,0.480364,0.475253,0.483361,0.487812,0.495704,0.499632,0.511905,0.526759,0.538898,0.551675,0.560402,0.565588,0.568385,0.570034,0.569091,0.567105,0.567547,0.564253,0.563045,0.557723,0.555482,0.552553,0.548610,0.546997,0.544078,0.540548,0.538835,0.537592,0.537332,0.536744,0.537376,0.537719,0.539435,0.541768,0.544255,0.548323,0.551964,...,1.585891,1.573518,1.562750,1.541807,1.531016,1.532603,1.506974,1.499881,1.485479,1.466208,1.458720,1.462966,1.438871,1.430821,1.423394,1.413873,1.396752,1.388556,1.376332,1.375741,1.379984,1.369669,1.368071,1.358944,1.349458,1.360884,1.361272,1.365245,1.375125,1.403202,1.397915,1.402013,1.432023,1.394996,1.336787,38.29,32.05,low,low,high
13117,ID_RL163URJ,0.569931,0.568551,0.572118,0.577033,0.579474,0.588586,0.596693,0.602761,0.608331,0.620611,0.634473,0.646010,0.654801,0.657721,0.659428,0.659293,0.657904,0.655353,0.652347,0.648569,0.646517,0.643732,0.639707,0.634865,0.633141,0.628174,0.626648,0.624459,0.623656,0.621238,0.620783,0.620620,0.623020,0.623431,0.625472,0.627468,0.632376,0.637180,0.639681,...,1.718914,1.711173,1.680911,1.670456,1.654504,1.632730,1.642662,1.614414,1.596063,1.590364,1.577994,1.554708,1.545799,1.531920,1.527434,1.519838,1.506132,1.494076,1.487568,1.483776,1.475529,1.472133,1.459305,1.449881,1.479502,1.455898,1.475418,1.458448,1.466081,1.458973,1.464456,1.506168,1.461038,1.502701,1.507115,36.89,22.33,ok,ok,ok
13118,ID_Q900J00W,0.490708,0.487741,0.490790,0.493822,0.499726,0.507649,0.516595,0.525130,0.530810,0.542568,0.557456,0.569906,0.577798,0.580788,0.581462,0.582782,0.582637,0.581751,0.577289,0.577422,0.574038,0.570820,0.565723,0.562900,0.560466,0.557981,0.555694,0.554035,0.553173,0.549626,0.549007,0.547638,0.548967,0.549563,0.550607,0.552420,0.554140,0.557201,0.560925,...,1.527578,1.506856,1.506530,1.494645,1.476220,1.466450,1.441356,1.423204,1.417612,1.414285,1.401067,1.381475,1.382106,1.354074,1.356723,1.356695,1.336694,1.324350,1.320942,1.310263,1.303583,1.306004,1.306437,1.303803,1.297072,1.293773,1.299638,1.304563,1.321738,1.317694,1.322799,1.333435,1.312727,1.341047,1.381044,42.81,41.05,low,ok,high
13119,ID_3USRZVNU,0.516280,0.517853,0.520625,0.526440,0.537743,0.54

In [8]:
def change_col(df,ss,nss):
    y=df[ss]
    ynum=[]
    for i in y:
        if i=="low":
            ynum.append(-1.0)
        if i=="high":
            ynum.append(1.0)
        if i=="ok":
            ynum.append(0.0)
    df[nss]=ynum
         

In [9]:
change_col(df,"hdl_cholesterol_human","hdl_val")

In [10]:
change_col(df,"hemoglobin(hgb)_human","hemo_val")

In [11]:
change_col(df,"cholesterol_ldl_human","col_val")

In [12]:
df.drop("hdl_cholesterol_human",axis=1,inplace=True)
df.drop("hemoglobin(hgb)_human",axis=1,inplace=True)
df.drop("cholesterol_ldl_human",axis=1,inplace=True)


In [13]:
df.iloc[:,1:]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_val,hemo_val,col_val
0,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,0.554599,...,1.469838,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,0.0,0.0,0.0
1,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,0.554408,...,1.552979,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,0.0,1.0,1.0
2,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,0.540124,...,1.516723,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,0.0,0.0,1.0
3,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,0.575452,...,1.442957,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,-1.0,0.0,1.0
4,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.557129,0.567030,0.577731,0.589192,0.604401,0.611372,0.614571,

In [14]:
df.describe()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_val,hemo_val,col_val
count,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,...,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000
mean,0.507347,0.509800,0.513687,0.519391,0.525976,0.533482,0.539453,0.544755,0.552902,0.565279,0.578415,0.590215,0.597083,0.600618,0.602828,0.603434,0.602075,0.599335,0.596222,0.593066,0.589816,0.586821,0.583665,0.580336,0.577325,0.574321,0.571803,0.569906,0.568239,0.566951,0.566204,0.565929,0.566327,0.567358,0.568966,0.571237,0.574091,0.577933,0.581767,0.585947,...,1.494436,1.481857,1.470147,1.458943,1.447029,1.436010,1.424403,1.413153,1.402854,1.394788,1.382297,1.373392,1.364888,1.356420,1.348587,1.341277,1.333918,1.327140,1.320583,1.314208,1.309417,1.305020,1.301597,1.299224,1.298162,1.297279,1.296103,1.292939,1.287715,1.279190,1.271131,1.264782,1.262234,1.274563,1.261993,41.088709,31.534744,-0.036530,0.027397,0.351598
std,0.042075,0.041587,0.041525,0.041676,0.041125,0.040644,0.040715,0.040946,0.040820,0.041038,0.041164,0.041268,0.041501,0.041560,0.041508,0.041379,0.041080,0.040716,0.040453,0.040368,0.040127,0.040101,0.040019,0.039927,0.039915,0.039767,0.039740,0.039792,0.039787,0.039715,0.039708,0.039738,0.039781,0.039860,0.039862,0.039979,0.040065,0.040266,0.040384,0.040327,...,0.073972,0.072704,0.071497,0.070351,0.069132,0.068116,0.066951,0.066179,0.065450,0.065161,0.063811,0.063188,0.062849,0.062148,0.061847,0.061448,0.060891,0.060457,0.060267,0.059903,0.059883,0.059731,0.059607,0.059692,0.060137,0.060583,0.062033,0.064989,0.069659,0.077427,0.085519,0.092525,0.097079,0.103298,0.102908,2.964205,8.118994,0.654157,0.329919,0.565089
min,0.237409,0.251987,0.269550,0.284636,0.302883,0.318334,0.333950,0.345124,0.361352,0.374106,0.388672,0.407299,0.416705,0.420315,0.423078,0.423848,0.423839,0.422310,0.421942,0.419578,0.416689,0.414667,0.413794,0.410312,0.408713,0.405934,0.403518,0.402689,0.401398,0.400746,0.399525,0.399710,0.399237,0.400620,0.403256,0.404547,0.407792,0.412233,0.416269,0.421089,...,1.258771,1.252074,1.243098,1.234149,1.228081,1.216854,1.205064,1.198276

In [15]:
minmax=MinMaxScaler()
minmax.fit(df.iloc[:,1:])

MinMaxScaler()

In [16]:
data_transform=minmax.transform(df.iloc[:,1:])

In [17]:
data_transform

array([[0.57629797, 0.56316263, 0.56058608, ..., 0.5       , 0.5       ,
        0.5       ],
       [0.55695348, 0.55489242, 0.54011688, ..., 0.5       , 1.        ,
        1.        ],
       [0.49382147, 0.5147261 , 0.45645531, ..., 0.5       , 0.5       ,
        1.        ],
       ...,
       [0.65491655, 0.66406548, 0.6553014 , ..., 0.5       , 0.5       ,
        0.5       ],
       [0.52177034, 0.5510518 , 0.50663204, ..., 0.5       , 0.5       ,
        0.5       ],
       [0.62079073, 0.62378824, 0.60670488, ..., 1.        , 0.5       ,
        1.        ]])

In [18]:
X=data_transform[:,:-3]
y=data_transform[:,-3:]

In [19]:
y

array([[0.5, 0.5, 0.5],
       [0.5, 1. , 1. ],
       [0.5, 0.5, 1. ],
       ...,
       [0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5],
       [1. , 0.5, 1. ]])

In [20]:
import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences

In [21]:
clear_session()



input_len=len(X[0])

print(input_len)
output_size=3
drop_frac0=0.0 
drop_frac1=0.0



input1=Input(shape=(input_len,),)

#flatt=Flatten()(lstm1)

non=42
#initializer = tf.keras.initializers.LecunNormal()
#initializer=tf.keras.initializers.LecunUniform()
#initializer=tf.keras.initializers.HeUniform(    seed=None)
#initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

initializer="normal"
d1=Dense(193,activation="sigmoid",kernel_initializer=initializer,input_dim=input_len)(input1)
d1=Dropout(drop_frac0)(d1)



pred=Dense(output_size,)(d1)

model = Model(inputs=input1, outputs=pred)

opt = tf.keras.optimizers.Adamax(learning_rate=0.01)


rmse = tf.keras.metrics.RootMeanSquaredError()
model.compile(loss="mean_squared_error",
    optimizer=opt,
    metrics=[rmse])

172


In [ ]:
model.fit(X,
          y,
          epochs=555, 
          batch_size=3,
          validation_split=0.1,
          verbose=1,
          
          shuffle=False
          
          )

Epoch 1/555
3942/3942 [==============================] - 16s 4ms/step - loss: 0.0662 - root_mean_squared_error: 0.2572 - val_loss: 0.0666 - val_root_mean_squared_error: 0.2580
Epoch 2/555
3942/3942 [==============================] - 16s 4ms/step - loss: 0.0650 - root_mean_squared_error: 0.2550 - val_loss: 0.0652 - val_root_mean_squared_error: 0.2553
Epoch 3/555
3942/3942 [==============================] - 16s 4ms/step - loss: 0.0637 - root_mean_squared_error: 0.2524 - val_loss: 0.0643 - val_root_mean_squared_error: 0.2536
Epoch 4/555
3942/3942 [==============================] - 16s 4ms/step - loss: 0.0626 - root_mean_squared_error: 0.2502 - val_loss: 0.0635 - val_root_mean_squared_error: 0.2519
Epoch 5/555
3942/3942 [==============================] - 16s 4ms/step - loss: 0.0616 - root_mean_squared_error: 0.2482 - val_loss: 0.0623 - val_root_mean_squared_error: 0.2496
Epoch 6/555
  62/3942 [..............................] - ETA: 13s - loss: 0.0586 - root_mean_squared_error: 0.2420